In [1]:
import os
current_dir = %pwd
project_dir = os.path.dirname(current_dir)
%cd $project_dir

d:\DATA\a.asri\doc-classifier


d:\DATA\a.asri\doc-classifier\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### 00 - Préambule ###
L'objectif de cette étape est la modélisation des données de texte sans structure (cf. notebook 08-2).
Les modèles ayant obtenu les scores les plus elevés ont été retenu pour la suite : 

- Logistic Regression
- Random Forest Classfier
- Nearest Centroid
- Extra Trees Classifier
- XGBClassifier
- LGBMClassifier

Le paramétrage des modèles choisis via les grilles de recherches (GridSearch) permettra ,
d'avoir des niveau de précision plus elevés, et permettre d'hiérarchiser encore plus chacun des modèles entraînés.

In [2]:
#Importation des bibliothèques nécessaires

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import load

#Chargement des données et préparation des données features et target :

df = pd.read_csv("data\processed\words_structure.csv") 
seed = 42
df['words'] = df['words'].fillna('')

target = df['category']
features = df.drop('category', axis=1)

In [3]:
# Répértoire des outputs
save_dir = "models"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

### 01 - Préparation des données

In [4]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=seed)

# Chargement du LabelEncoder préalablement sauvegardé
le = load("models/target_LabelEncoder.joblib")

"""Selon l'encoder enregistré : 
    email :                     0
    handwritten :               1
    invoice :                   2
    national_identity_card :    3
    passeport :                 4
    scientific_publication :    5
"""

# Chargement du TfidfVectorizer préalablement sauvegardé
tfidf_vectorizer = load('models/TfidfVectorizer.joblib')

In [5]:
# Encodage des étiquettes de classe
y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

# Extraction des données textuelles et conversion en listes
X_train_corpus = X_train['words'].tolist()
X_test_corpus = X_test['words'].tolist()

In [6]:
# Transformation des données textuelles en vecteurs TF-IDF
X_train_tfidf = tfidf_vectorizer.transform(X_train_corpus)
X_test_tfidf = tfidf_vectorizer.transform(X_test_corpus)

In [7]:
occurrences = df['category'].value_counts()
print(occurrences)

print(" \nNous constatons que le jeu de données est plus au moins équilibré sauf pour la catégorie 'national_identity_card', \n \
nous pourrions utiliser L'Over Sampling pour équilibrer encore plus notre je de données")

category
invoice                   1770
passeport                 1397
handwritten               1250
email                     1250
scientific_publication    1250
national_identity_card     192
Name: count, dtype: int64
 
Nous constatons que le jeu de données est plus au moins équilibré sauf pour la catégorie 'national_identity_card', 
 nous pourrions utiliser L'Over Sampling pour équilibrer encore plus notre je de données


In [8]:
import joblib

# Enregistrer le modèle dans le répertoire "models"
save_dir = "models"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

### 02 - Modélisation des données

Les modèles retenus suite aux résultats de la bibliothèque "lazypredict" sont :

| Modèle                   | Accuracy | Balanced Accuracy | ROC AUC | F1 Score | Time Taken (s) |
|--------------------------|----------|-------------------|---------|----------|----------------|
| LogisticRegression       | 0.86     | 0.82              | None    | 0.86     | 95.99          |
| NearestCentroid          | 0.85     | 0.81              | None    | 0.85     | 95.60          |
| ExtraTreesClassifier     | 0.86     | 0.81              | None    | 0.86     | 256.07         |
| RandomForestClassifier   | 0.84     | 0.79              | None    | 0.84     | 190.07         |
| XGBClassifier            | 0.82     | 0.77              | None    | 0.82     | 322.53         |
| LGBMClassifier           | 0.82     | 0.76              | None    | 0.82     | 98.41          |

Ces modèles ont été sélectionnés en fonction de leurs performances en termes de précision (Accuracy) et de scores F1, ainsi que de leur temps d'exécution.

In [10]:
import os
import time
import warnings
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Initialiser un DataFrame pour stocker les résultats
results_df = pd.DataFrame(columns=['Model', 'Best Parameters', 'Best Accuracy', 'Time Taken'])

def train_and_evaluate_model(model, params, X_train, y_train, X_test, y_test, model_name):
    print(f"Training {model_name}...")
    start_time = time.time()
    clf = GridSearchCV(model, params, scoring='accuracy', cv=5, n_jobs=-1)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        clf.fit(X_train, y_train)
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # Enregistrement des paramètres et des résultats dans le DataFrame
    results_df.loc[len(results_df)] = [model_name, clf.best_params_, clf.best_score_, elapsed_time]
    
    # Création du sous-répertoire clf_reports et clf_imb_reports s'ils n'existent pas déjà
    clf_reports_dir = os.path.join(save_dir, "clf_reports")
    if not os.path.exists(clf_reports_dir):
        os.makedirs(clf_reports_dir)
    
    # Sauvegarde du rapport de classification sur les données de test
    y_pred = clf.predict(X_test)
    report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
    report_path = os.path.join(clf_reports_dir, f"{model_name}_classification_report_{time.strftime('%Y%m%d%H%M%S')}.csv")
    report_df.to_csv(report_path)
    print(f"Classification report saved at: {report_path}")
    


          02 - 1 Préparation des paramètres pour la grille de recherche GridSearchCV

In [11]:
#Affichage des modèles entraînes (147 combinaisons de modèles environ)
from itertools import product
import pandas as pd
from sklearn.model_selection import ParameterGrid

# Définition de tous les paramètres possibles pour chaque modèle
lr_params = {'C': [100, 10], 'solver': ['lbfgs'], 'penalty': ['elasticnet', 'l2']}
nc_params = {'metric': ['euclidean'], 'shrink_threshold': [None, 0.5]}
etc_params = {'criterion': ['gini', 'entropy'], 'n_estimators': [100, 200], 'max_depth': [None, 30], 'min_samples_split': [2], 'max_features': ['auto', 'sqrt', 'log2'], 'bootstrap': [True], 'oob_score': [True, False]}
rfc_params = {'criterion': ['gini', 'entropy'], 'n_estimators': [200, 100, 500], 'max_depth': [None, 30], 'min_samples_split': [5], 'max_features': ['auto', 'sqrt', 'log2'], 'bootstrap': [True], 'oob_score': [True, False]}
xgb_params = {'n_estimators': [200, 100, 300], 'max_depth': [5], 'learning_rate': [0.1], 'gamma': [0.2],  'reg_alpha': [0, 0.5],'reg_lambda': [0, 0.5]}
lgbm_params = {'boosting_type': ['gbdt', 'dart'], 'n_estimators': [200, 100], 'max_depth': [10, 5], 'learning_rate': [0.1],
                'min_child_samples': [10, 50], 'class_weight': [None, 'balanced'], 'num_leaves': [31], 'force_row_wise': [True], 'force_col_wise': [False]}

# Création d'un DataFrame pour afficher toutes les combinaisons de paramètres
params_combinations = []

for model, params in zip(['LogisticRegression', 'NearestCentroid', 'ExtraTreesClassifier', 'RandomForestClassifier', 'XGBClassifier', 'LGBMClassifier'], 
                         [lr_params, nc_params, etc_params, rfc_params, xgb_params, lgbm_params]):
    param_grid = list(ParameterGrid(params))
    params_combinations.extend([(model, param) for param in param_grid])

params_df = pd.DataFrame(params_combinations, columns=['Model', 'Parameters'])
params_df


,Model,Parameters
0,LogisticRegression,"{'C': 100, 'penalty': 'elasticnet', 'solver': ..."
1,LogisticRegression,"{'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}"
2,LogisticRegression,"{'C': 10, 'penalty': 'elasticnet', 'solver': '..."
3,LogisticRegression,"{'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}"
4,NearestCentroid,"{'metric': 'euclidean', 'shrink_threshold': None}"
...,...,...
165,LGBMClassifier,"{'boosting_type': 'dart', 'class_weight': 'bal..."
166,LGBMClassifier,"{'boosting_type': 'dart', 'class_weight': 'bal..."
167,LGBMClassifier,"{'boosting_type': 'dart', 'class_weight': 'bal..."
168,LGBMClassifier,"{'boosting_type': 'dart', 'class_weight': 'bal..."


In [12]:
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Instantiation des modèles : 
lr = LogisticRegression()
nc = NearestCentroid()
etc = ExtraTreesClassifier()
rfc = RandomForestClassifier()
xgb = XGBClassifier()
lgbm = LGBMClassifier()

# Entraîner et évaluer chaque modèle
train_and_evaluate_model(lr, lr_params, X_train_tfidf, y_train_encoded, X_test_tfidf, y_test_encoded, "LogisticRegression")
train_and_evaluate_model(nc, nc_params, X_train_tfidf, y_train_encoded, X_test_tfidf, y_test_encoded, "NearestCentroid")
train_and_evaluate_model(etc, etc_params, X_train_tfidf, y_train_encoded, X_test_tfidf, y_test_encoded, "ExtraTreesClassifier")
train_and_evaluate_model(rfc, rfc_params, X_train_tfidf, y_train_encoded, X_test_tfidf, y_test_encoded, "RandomForestClassifier")
train_and_evaluate_model(xgb, xgb_params, X_train_tfidf, y_train_encoded, X_test_tfidf, y_test_encoded, "XGBClassifier")
train_and_evaluate_model(lgbm, lgbm_params, X_train_tfidf, y_train_encoded, X_test_tfidf, y_test_encoded, "LGBMClassifier")


suffix_num = 1
while os.path.exists(os.path.join(save_dir, f"best_models_results_{suffix_num}.csv")):
    suffix_num += 1

# Enregistrer les résultats dans un fichier CSV avec le suffixe numérique
results_filename = f"best_models_results_{suffix_num}.csv"
results_path = os.path.join(save_dir, results_filename)
results_df.to_csv(results_path, index=False)
print(f"Results saved at: {results_path}")


Training LogisticRegression...
Classification report saved at: models\clf_reports\LogisticRegression_classification_report_20240515220228.csv
Training NearestCentroid...
Classification report saved at: models\clf_reports\NearestCentroid_classification_report_20240515220229.csv
Training ExtraTreesClassifier...
Classification report saved at: models\clf_reports\ExtraTreesClassifier_classification_report_20240515221254.csv
Training RandomForestClassifier...
Classification report saved at: models\clf_reports\RandomForestClassifier_classification_report_20240515223644.csv
Training XGBClassifier...
Classification report saved at: models\clf_reports\XGBClassifier_classification_report_20240515230013.csv
Training LGBMClassifier...
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightG

In [13]:
import os
import pandas as pd

# Chemin vers le répertoire contenant les fichiers de classification report

clf_reports_dir = os.path.join(save_dir, "clf_reports")
if not os.path.exists(clf_reports_dir):
    os.makedirs(clf_reports_dir)

# Initialisation d'une liste pour stocker les DataFrames de classification reports
dfs = []

# Parcourir tous les fichiers CSV dans le répertoire
for filename in os.listdir(clf_reports_dir):
    if filename.endswith(".csv"):
        # Lire le fichier CSV dans un DataFrame
        df = pd.read_csv(os.path.join(clf_reports_dir, filename))
        
        # Extraire le nom du modèle à partir du nom du fichier
        model_name = os.path.splitext(filename)[0]
        
        # Ajouter le nom du modèle comme nouvelle colonne dans le DataFrame
        df['Model'] = model_name
        
        # Ajouter le DataFrame à la liste
        dfs.append(df)

# Concaténer tous les DataFrames de classification reports en un seul DataFrame
combined_df = pd.concat(dfs, ignore_index=True)


# Afficher le DataFrame combiné
results_path = os.path.join(save_dir, "classification_reports_combined.csv")

combined_df.to_csv(results_path, index=False)




In [14]:
combined_df

,Unnamed: 0,precision,recall,f1-score,support,Model
0,0,0.991667,0.904943,0.946322,263.000000,ExtraTreesClassifier_classification_report_202...
1,1,0.767442,0.584071,0.663317,226.000000,ExtraTreesClassifier_classification_report_202...
2,2,0.941504,0.896552,0.918478,377.000000,ExtraTreesClassifier_classification_report_202...
3,3,0.952381,0.454545,0.615385,44.000000,ExtraTreesClassifier_classification_report_202...
4,4,0.632242,0.972868,0.766412,258.000000,ExtraTreesClassifier_classification_report_202...
5,5,0.961373,0.881890,0.919918,254.000000,ExtraTreesClassifier_classification_report_202...
6,accuracy,0.845992,0.845992,0.845992,0.845992,ExtraTreesClassifier_classification_report_202...
7,macro avg,0.874435,0.782478,0.804972,1422.000000,ExtraTreesClassifier_classification_report_202...
8,weighted avg,0.870893,0.845992,0.846364,1422.000000,ExtraTreesClassifier_classification_report_202...
9,0,0.971660,0.912548,0.941176,263.000000,LGBMClassifier_classification_report_202405152...
